In [2]:
import sys
sys.path.append('./')

import TSP
import math
import random

n = 100

# Create n random points

points = [(random.randint(0, 100), random.randint(0, 100)) for i in range(n)]

# Dictionary of Euclidean distance between each pair of points

dist = {(i, j): math.sqrt(sum((points[i][k]-points[j][k])**2 for k in range(2)))  for i in range(n) for j in range(i)}

print(TSP.main(n,dist))

([0, 3, 28, 17, 94, 98, 33, 13, 56, 19, 12, 88, 86, 74, 29, 25, 32, 70, 63, 16, 57, 58, 47, 48, 99, 69, 1, 41, 80, 95, 31, 71, 7, 5, 78, 8, 92, 73, 85, 2, 60, 93, 87, 55, 14, 81, 96, 54, 89, 75, 6, 72, 23, 52, 61, 45, 40, 34, 20, 77, 46, 11, 44, 36, 67, 15, 90, 42, 53, 82, 66, 51, 10, 39, 76, 22, 26, 50, 65, 84, 35, 49, 37, 59, 64, 18, 27, 9, 68, 43, 97, 4, 38, 24, 83, 62, 79, 91, 30, 21], 806.028853788602)


In [66]:
# 副問題のノード
# 全点よりダイクストラをして、他の点への最短ルートと最短距離を記録

from heapq import*
import pandas as pd

def DIJKSTRA(point,d):
    cost      = [float("inf")]*n
    path      = [[]for i in range(n)]
    Q         = []
    cost[point]=0
    path[point]=[point]
    heappush(Q,(point,cost[point],path[point]))
    while Q:
        prefnd,tmpC,tmpP=heappop(Q)
        if cost[prefnd]<tmpC:
            continue
        for node,c in d[prefnd]:
            altC=c+tmpC
            altP=tmpP+[node]
            if cost[node]>altC:
                path[node]=altP
                cost[node]=altC
                heappush(Q,(node,altC,altP))
    return cost,path

df=pd.read_excel("./warehouse_node.xlsx").fillna("")

n=max(df["node_number"])+1
d=[[]for i in range(n)]

nodelist=pd.concat([df.loc[:,"node_number"],df.loc[:,"connect_node_1":"connect_node_4"]],axis=1).values.tolist()

d={ line[0]:[int(num) for num in line[1:] if type(num)!=str] for line in nodelist}
d={k:[(i,1)for i in v] for k,v in d.items()}

toexcel=[["start","end","cost","path"]]
for start in range(n):
    c,p=DIJKSTRA(start,d)
    for end,cost,path in zip(range(n),c,p):
        toexcel.append([start,end,cost,",".join(map(str,path))])

pd.DataFrame(toexcel).to_excel("./warehouse_path.xlsx",header=None,index=None)